<a href="https://colab.research.google.com/github/ElenaShargina/nnetworks/blob/main/(1)_%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_%D0%B8%D1%81%D0%BA%D1%83%D1%81%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%B0_%D0%B8_%D0%B3%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%BF%D1%83%D1%81%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Реализована работа базовой модели искусственного нейрона Мак-Каллока – Питтса.

Функция активация - сигмоидальная.
Корректировка весов происходит с помощью стохастического градиентного спуска.

# Подготовка данных

In [1]:
# Импортируем необходимые пакеты
import numpy as np

In [2]:
# количество элементов в обучающей выборке
m = 17
# величина шага
h = 2
# количество эпох без улучшения точности, после которого остановим поиск
patience = 10

# точность
eps = 0.000001

# обучающая выборка
data_x = np.array([
    [0.1, 0.2, 0.3, 0.4],
    [0.2, 0.3, 0.4, 0.5],
    [0.3, 0.4, 0.5, 0.6],
    [0.4, 0.5, 0.6, 0.7],
    [0.5, 0.6, 0.7, 0.8],
    [0.6, 0.7, 0.8, 0.9],
    [0.7, 0.8, 0.9, 0],
    [0.8, 0.9, 0, -0.1],
    [0.9, 0, -0.1, -0.2],
    [0, -0.1, -0.2, -0.3],
    [-0.1, -0.2, -0.3, -0.4],
    [-0.2, -0.3, -0.4, -0.5],
    [-0.3, -0.4, -0.5, -0.6],
    [-0.4, -0.5, -0.6, -0.7],
    [-0.5, -0.6, -0.7, -0.8],
    [-0.6, -0.7, -0.8, -0.9],
    [-0.7, -0.8, -0.9, 0]
])
# правильные ответы к обучающей выборке
data_y = np.array([1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 0, -1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7])


Задаем первичные веса - какие-то маленькие значения.

In [3]:
weights = np.array([0.1, 0.2, 0.3, 0.4])

# Задание функций

Функция активации - сигмоидная

$\phi(x) = \frac{1}{1+e^{-x}}$

In [4]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

Производная от сигмоидной функции

$ \phi^\prime(x) = \frac{e^{-x}}{(1+e^{-x})^2}$

In [5]:
def diffsig(x): 
    return np.exp(-x)/(1+np.exp(-x))**2

Функция потерь (MSE)

$E(w_1,...,w_n)=\frac{\sum_{i=1}^m{(y-\phi(\sum_{i=1}^n{w_ix_i}))^2}}{m}$

In [6]:
def loss_func(weights, data_y, data_x):
    return np.sum((data_y - sigmoid(np.sum(data_x*weights)))**2)/weights.size

Производная функции потерь по j-му весу

$\frac{\partial E}{\partial w_j} = -{\frac{2}{m}(y-\phi({\sum_{i=1}^n}{w_ix_i}))\phi\prime({\sum_{i=1}^n}{w_ix_i})x_j}$

In [7]:
def diff_loss_func(j, weights, y, x): 
    return -2 * (y - sigmoid(np.sum(x*weights))) * diffsig(np.sum(x*weights)) * x[j]/weights.size

# Работа алгоритма

Стохастический градиентный спуск - после анализа каждого элемента выборки будем менять веса модели по формуле
$w_i=w_i-h\frac{\partial E}{\partial w_i}$


In [8]:
# счетчик итераций без улучшений, 
# заканчиваем цикл, если он достигнет параметра patience
failure_stop = 0
# порядок просмотра элементов выборки,
# будем перемешивать его каждую эпоху
index = np.arange(m)

for k in range(4000):
    # Перемешиваем обучающую выборку
    np.random.shuffle(index)
    # loss_old - значение функции потерь на предыдущей итерации, 
    # на первой итерации не существует
    if k>0:
      loss_old = loss_new

    # Изменяем веса после анализа каждого элемента выборки
    for i in index:
        for j in range(weights.size):
          weights[j] = weights[j] - h*diff_loss_func(j, weights, data_y[i], data_x[i])

    # высчитываем функцию потерь на новых весах
    loss_new = loss_func(weights, data_y, data_x)

    # если итерация не первая, то проверяем, улучшилась ли точность
    if k>0 and abs(loss_new - loss_old) < eps:
        failure_stop += 1
    else:
        failure_stop = 0

    # если точность не улучшилась уже заданное количество итераций, то останавливаем цикл    
    if failure_stop == patience:
        print(f'Количество итераций без улучшений: {failure_stop}, останавливаем работу алгоритма')
        break

# выводим результирующие данные
print(f'Получившиеся значения весов: {weights.round(6)}')
print(f'Количество итераций алгоритма: {k}')
print(f'Значение функции потерь на обучающей выборке: {loss_new:.6f}')

Количество итераций без улучшений: 10, останавливаем работу алгоритма
Получившиеся значения весов: [7.126878 4.067194 4.387924 4.702111]
Количество итераций алгоритма: 202
Значение функции потерь на обучающей выборке: 11.007412


In [11]:
for i in range(data_y.size):
  print(np.round(np.sum(data_x[i]*weights),2), '---', data_y[i])

4.72 --- 1.0
6.75 --- 1.2
8.78 --- 1.3
10.81 --- 1.4
12.84 --- 1.5
14.87 --- 1.6
12.19 --- 1.7
8.89 --- 1.8
5.03 --- 1.9
-2.69 --- 0.0
-4.72 --- -1.0
-6.75 --- -1.2
-8.78 --- -1.3
-10.81 --- -1.4
-12.84 --- -1.5
-14.87 --- -1.6
-12.19 --- -1.7
